## Installation and mount Google Drive

In [ ]:
!pip install --quiet transformers==2.9.0
!pip install --quiet nltk==3.4.5

In [ ]:
# connect your personal google drive to store the trained model
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Generate distractors (wrong choices) for MCQ options

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

sentence1 = "Srivatsan loves to watch cricket during his free time"
sentence2 = "Srivatsan is annoyed by a cricket in his room"

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# An example of a word with two different senses
original_word = "cricket"

syns = wn.synsets(original_word,'n')

for syn in syns:
  print (syn, ": ",syn.definition(),"\n" )

Synset('cricket.n.01') :  leaping insect; male makes chirping noises by rubbing the forewings together 

Synset('cricket.n.02') :  a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs 



In [ ]:
# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors


synset_to_use = wn.synsets(original_word,'n')[0]
distractors_calculated = get_distractors_wordnet(synset_to_use,original_word)

print ("\noriginal word: ",original_word.capitalize())
print (distractors_calculated)


original_word = "cricket"
synset_to_use = wn.synsets(original_word,'n')[1]
distractors_calculated = get_distractors_wordnet(synset_to_use,original_word)

print ("\noriginal word: ",original_word.capitalize())
print (distractors_calculated)


original word:  Cricket
['Grasshopper']

original word:  Cricket
['Ball Game', 'Field Hockey', 'Football', 'Hurling', 'Lacrosse', 'Polo', 'Pushball', 'Ultimate Frisbee']


## Download pretrained BERT WSD Model and extract

Download pre-trained BERT WSD from [here](https://entuedu-my.sharepoint.com/:f:/g/personal/boonpeng001_e_ntu_edu_sg/EiWzblOyyOBDtuO3klUbXoAB3THFzke-2MLWguIXrDopWg?e=08umXD)

Click the download button at the top left of the link to download a file named "bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6.zip"

Place the zip file in your Google drive home folder

In [ ]:
import os
import zipfile

bert_wsd_pytorch = "/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6.zip"
extract_directory = "/content/gdrive/My Drive"

extracted_folder = bert_wsd_pytorch.replace(".zip","")

#  If unzipped folder exists don't unzip again.
if not os.path.isdir(extracted_folder):
  with zipfile.ZipFile(bert_wsd_pytorch, 'r') as zip_ref:
      zip_ref.extractall(extract_directory)
else:
  print (extracted_folder," is extracted already")

/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6  is extracted already


## Find the correct sense (contextual meaning) of a given word in a sentence

In [ ]:
import torch
import math
from transformers import BertModel, BertConfig, BertPreTrainedModel, BertTokenizer

class BertWSD(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.ranking_linear = torch.nn.Linear(config.hidden_size, 1)

        self.init_weights()


# def _forward(args, model, batch):
#     batch = tuple(t.to(args.device) for t in batch)
#     outputs = model.bert(input_ids=batch[0], attention_mask=batch[1], token_type_ids=batch[2])

#     return model.dropout(outputs[1])
    

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = "/content/gdrive/My Drive/bert_base-augmented-batch_size=128-lr=2e-5-max_gloss=6"


model = BertWSD.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)
# add new special token
if '[TGT]' not in tokenizer.additional_special_tokens:
    tokenizer.add_special_tokens({'additional_special_tokens': ['[TGT]']})
    assert '[TGT]' in tokenizer.additional_special_tokens
    model.resize_token_embeddings(len(tokenizer))
    
model.to(DEVICE)
model.eval()

BertWSD(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30523, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [ ]:
import csv
import os
from collections import namedtuple

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

import torch
from tqdm import tqdm

GlossSelectionRecord = namedtuple("GlossSelectionRecord", ["guid", "sentence", "sense_keys", "glosses", "targets"])
BertInput = namedtuple("BertInput", ["input_ids", "input_mask", "segment_ids", "label_id"])



def _create_features_from_records(records, max_seq_length, tokenizer, cls_token_at_end=False, pad_on_left=False,
                                  cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                  sequence_a_segment_id=0, sequence_b_segment_id=1,
                                  cls_token_segment_id=1, pad_token_segment_id=0,
                                  mask_padding_with_zero=True, disable_progress_bar=False):
    """ Convert records to list of features. Each feature is a list of sub-features where the first element is
        always the feature created from context-gloss pair while the rest of the elements are features created from
        context-example pairs (if available)
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """
    features = []
    for record in tqdm(records, disable=disable_progress_bar):
        tokens_a = tokenizer.tokenize(record.sentence)

        sequences = [(gloss, 1 if i in record.targets else 0) for i, gloss in enumerate(record.glosses)]

        pairs = []
        for seq, label in sequences:
            tokens_b = tokenizer.tokenize(seq)

            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

            # The convention in BERT is:
            # (a) For sequence pairs:
            #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
            #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
            #
            # Where "type_ids" are used to indicate whether this is the first
            # sequence or the second sequence. The embedding vectors for `type=0` and
            # `type=1` were learned during pre-training and are added to the wordpiece
            # embedding vector (and position vector). This is not *strictly* necessary
            # since the [SEP] token unambiguously separates the sequences, but it makes
            # it easier for the model to learn the concept of sequences.
            #
            # For classification tasks, the first vector (corresponding to [CLS]) is
            # used as as the "sentence vector". Note that this only makes sense because
            # the entire model is fine-tuned.
            tokens = tokens_a + [sep_token]
            segment_ids = [sequence_a_segment_id] * len(tokens)

            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

            if cls_token_at_end:
                tokens = tokens + [cls_token]
                segment_ids = segment_ids + [cls_token_segment_id]
            else:
                tokens = [cls_token] + tokens
                segment_ids = [cls_token_segment_id] + segment_ids

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding_length = max_seq_length - len(input_ids)
            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            pairs.append(
                BertInput(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_id=label)
            )

        features.append(pairs)

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import re
import torch
from tabulate import tabulate
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import BertTokenizer
import time


MAX_SEQ_LENGTH = 128

def get_sense(sent):
  re_result = re.search(r"\[TGT\](.*)\[TGT\]", sent)
  if re_result is None:
      print("\nIncorrect input format. Please try again.")

  ambiguous_word = re_result.group(1).strip()

  results = dict()

  wn_pos = wn.NOUN
  for i, synset in enumerate(set(wn.synsets(ambiguous_word, pos=wn_pos))):
      results[synset] =  synset.definition()

  if len(results) ==0:
    return (None,None,ambiguous_word)

  # print (results)
  sense_keys=[]
  definitions=[]
  for sense_key, definition in results.items():
      sense_keys.append(sense_key)
      definitions.append(definition)


  record = GlossSelectionRecord("test", sent, sense_keys, definitions, [-1])

  features = _create_features_from_records([record], MAX_SEQ_LENGTH, tokenizer,
                                            cls_token=tokenizer.cls_token,
                                            sep_token=tokenizer.sep_token,
                                            cls_token_segment_id=1,
                                            pad_token_segment_id=0,
                                            disable_progress_bar=True)[0]

  with torch.no_grad():
      logits = torch.zeros(len(definitions), dtype=torch.double).to(DEVICE)
      # for i, bert_input in tqdm(list(enumerate(features)), desc="Progress"):
      for i, bert_input in list(enumerate(features)):
          logits[i] = model.ranking_linear(
              model.bert(
                  input_ids=torch.tensor(bert_input.input_ids, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  attention_mask=torch.tensor(bert_input.input_mask, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  token_type_ids=torch.tensor(bert_input.segment_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
              )[1]
          )
      scores = softmax(logits, dim=0)

      preds = (sorted(zip(sense_keys, definitions, scores), key=lambda x: x[-1], reverse=True))


  # print (preds)
  sense = preds[0][0]
  meaning = preds[0][1]
  return (sense,meaning,ambiguous_word)


sentence1 = "Srivatsan loves to watch **cricket** during his free time"


sentence_for_bert = sentence1.replace("**"," [TGT] ")
sentence_for_bert = " ".join(sentence_for_bert.split())
sense,meaning,answer = get_sense(sentence_for_bert)

print (sentence1)
print (sense)
print (meaning)

sentence2 = "Srivatsan is annoyed by a **cricket** in his room"
sentence_for_bert = sentence2.replace("**"," [TGT] ")
sentence_for_bert = " ".join(sentence_for_bert.split())
sense,meaning,answer = get_sense(sentence_for_bert)

print ("\n-------------------------------")
print (sentence2)
print (sense)
print (meaning)



Srivatsan loves to watch **cricket** during his free time
Synset('cricket.n.02')
a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs

-------------------------------
Srivatsan is annoyed by a **cricket** in his room
Synset('cricket.n.01')
leaping insect; male makes chirping noises by rubbing the forewings together


## Generate a question using context and answer with T5

![picture](https://drive.google.com/uc?export=view&id=1Dc6W3F__okw1q6GxhKs46lvgeeBsP0iG)


In [ ]:
from transformers import T5ForConditionalGeneration,T5Tokenizer

question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('t5-base')

def get_question(sentence,answer):
  text = "context: {} answer: {} </s>".format(sentence,answer)
  print (text)
  max_len = 256
  encoding = question_tokenizer.encode_plus(text,max_length=max_len, pad_to_max_length=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = question_model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=200)


  dec = [question_tokenizer.decode(ids) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question


sentence1 = "Srivatsan loves to watch **cricket** during his free time"
sentence2 = "Srivatsan is annoyed by a **cricket** in his room"


answer = "cricket"

sentence_for_T5 = sentence1.replace("**"," ")
sentence_for_T5 = " ".join(sentence_for_T5.split()) 
ques = get_question(sentence_for_T5,answer)
print (ques)


print ("\n**************************************\n")
sentence_for_T5 = sentence2.replace("**"," ")
sentence_for_T5 = " ".join(sentence_for_T5.split()) 
ques = get_question(sentence_for_T5,answer)
print (ques)


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

context: Srivatsan loves to watch cricket during his free time answer: cricket </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What sport does Srivatsan enjoy watching?

**************************************

context: Srivatsan is annoyed by a cricket in his room answer: cricket </s>
What insect is in Srivatsan's room?


## Putting it all together

In [ ]:
def getMCQs(sent):
  sentence_for_bert = sent.replace("**"," [TGT] ")
  sentence_for_bert = " ".join(sentence_for_bert.split())
  # try:
  sense,meaning,answer = get_sense(sentence_for_bert)
  if sense is not None:
    distractors = get_distractors_wordnet(sense,answer)
  else: 
    distractors = ["Word not found in Wordnet. So unable to extract distractors."]
  sentence_for_T5 = sent.replace("**"," ")
  sentence_for_T5 = " ".join(sentence_for_T5.split()) 
  ques = get_question(sentence_for_T5,answer)
  return ques,answer,distractors,meaning



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence1)
print (question)
print (answer)
print (distractors)
print (meaning)

print ("\n")
question,answer,distractors,meaning = getMCQs(sentence2)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Srivatsan loves to watch cricket during his free time answer: cricket </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What sport does Srivatsan enjoy watching?
cricket
['Ball Game', 'Field Hockey', 'Football', 'Hurling', 'Lacrosse', 'Polo', 'Pushball', 'Ultimate Frisbee']
a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs


context: Srivatsan is annoyed by a cricket in his room answer: cricket </s>
What insect is in Srivatsan's room?
cricket
['Grasshopper']
leaping insect; male makes chirping noises by rubbing the forewings together


In [ ]:
sentence = "The eating up of metals by the action of air, moisture or chemical on the metal’s surface is called **corrosion**. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The eating up of metals by the action of air, moisture or chemical on the metal’s surface is called corrosion . answer: corrosion </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the process of eating up metals called?
corrosion
['Decrepitude', 'Devastation', 'Wear']
a state of deterioration in metals caused by oxidation or chemical action


In [ ]:
sentence = " Oxygen is a member of the **chalcogen group** on the periodic table and is a highly reactive nonmetal and oxidizing agent that readily forms compounds (notably oxides) with most elements."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Oxygen is a member of the chalcogen group on the periodic table and is a highly reactive nonmetal and oxidizing agent that readily forms compounds (notably oxides) with most elements. answer: chalcogen group </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What group is oxygen a member of?
chalcogen group
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "The substances which take part and undergo change during a chemical reaction are called **reactants**."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The substances which take part and undergo change during a chemical reaction are called reactants . answer: reactants </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Substances that undergo change during a chemical reaction are called what?
reactants
['Alar', 'Bactericide', 'Carbon Tetrahalide', 'Carrier', 'Compound', 'Congener', 'Explosive', 'Fertilizer', 'Flux', 'Fraction', 'Fumigant', 'Herbicide', 'Intermediate', 'Larvicide', 'Pesticide', 'Phytochemical', 'Product', 'Reactant', 'Restrainer', 'Softener', 'Soil Conditioner', 'Stabilizer']
a chemical substance that is present at the start of a chemical reaction


In [ ]:
sentence = "The new substances produced in a chemical reaction are called **products**."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The new substances produced in a chemical reaction are called products . answer: products </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What are new substances produced in a chemical reaction called?
products
['Art', 'Classic', 'Composition', 'Improvisation', 'Invention', 'Master', 'Needlework', 'Piece', 'Product', 'Remake', 'Representation']
an artifact that has been created by someone or some process


In [ ]:
sentence = "process of production of heat is known as **exothermic reaction.**"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: process of production of heat is known as exothermic reaction. answer: exothermic reaction. </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the process of production of heat known as?
exothermic reaction.
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "**Rancidity** is the process where fats and oils in food get oxidized on exposure to air."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Rancidity is the process where fats and oils in food get oxidized on exposure to air. answer: Rancidity </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Fats and oils in food get oxidized on exposure to air?
Rancidity
['Decomposition', 'Deterioration', 'Putrefaction']
the state of being rancid; having a rancid scent or flavor (as of old cooking oil)


In [ ]:
sentence = "When an acid is been reacted with a metal, then **a salt and hydrogen gas** is formed."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: When an acid is been reacted with a metal, then a salt and hydrogen gas is formed. answer: a salt and hydrogen gas </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is formed when an acid is reacted with a metal?
a salt and hydrogen gas
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "**carbon dioxide** gas is been formed in the form of brisk effervescence"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: carbon dioxide gas is been formed in the form of brisk effervescence answer: carbon dioxide </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What gas is formed in the form of effervescence?
carbon dioxide
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "An acid is a substance which dissociates (or ionises) on dissolving in water to produce **hydrogen** ions"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: An acid is a substance which dissociates (or ionises) on dissolving in water to produce hydrogen ions answer: hydrogen </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What ions does an acid produce?
hydrogen
['Actinium', 'Allotrope', 'Argon', 'Arsenic', 'Astatine', 'Bohrium', 'Boron', 'Bromine', 'Carbon', 'Chlorine', 'Darmstadtium', 'Dubnium', 'Fluorine', 'Germanium', 'Hassium', 'Helium', 'Iodine', 'Krypton', 'Lawrencium', 'Meitnerium', 'Mendelevium', 'Metallic Element', 'Neon', 'Nitrogen', 'Nobelium', 'Noble Gas', 'Nonmetal', 'Oxygen', 'Phosphorus', 'Plutonium', 'Radon', 'Roentgenium', 'Rutherfordium', 'Seaborgium', 'Selenium', 'Silicon', 'Sulfur', 'Tellurium', 'Trace Element', 'Transactinide', 'Transuranic Element', 'Ununbium', 'Ununhexium', 'Ununpentium', 'Ununquadium', 'Ununtrium', 'Xenon']
a nonmetallic univalent element that is normally a colorless and odorless highly flammable diatomic gas; the simplest and lightest and most abundant element in the universe


In [ ]:
sentence = "The presence of charged particles i.e. ions in aqueous solution leads to **electric conductivity**."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The presence of charged particles i.e. ions in aqueous solution leads to electric conductivity . answer: electric conductivity </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


The presence of charged particles in aqueous solution leads to what?
electric conductivity
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "**Sulphuric acid** is used in the manufacture of fertilizers like ammonium sulphate, paints, dyes, chemicals, plastics, synthetic fibres, detergents, explosives and car batteries."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Sulphuric acid is used in the manufacture of fertilizers like ammonium sulphate, paints, dyes, chemicals, plastics, synthetic fibres, detergents, explosives and car batteries. answer: Sulphuric acid </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is used in the manufacture of paints and dyes?
Sulphuric acid
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "Acids are chemical substances which have a **sour** taste in aqueous medium."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Acids are chemical substances which have a sour taste in aqueous medium. answer: sour </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the taste of acid in aqueous medium?
sour
['Bitterness', 'Palatability', 'Rancidness', 'Sourness', 'Spiciness', 'Sweetness', 'Unpalatability']
the property of being acidic


In [ ]:
sentence = "Acids change the colour of blue litmus to **red**."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Acids change the colour of blue litmus to red . answer: red </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What color does acid change blue litmus to?
red
['Blond', 'Blue', 'Brown', 'Complementary Color', 'Green', 'Olive', 'Orange', 'Pastel', 'Pink', 'Purple', 'Salmon', 'Yellow']
red color or pigment; the chromatic color resembling the hue of blood


In [ ]:
sentence = "Acids have a pH **less than** 7"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Acids have a pH less than 7 answer: less than </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the pH of an acid?
less than
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "Bases have **bitter** taste."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Bases have bitter taste. answer: bitter </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What kind of taste do bases have?
bitter
['Bitterness', 'Palatability', 'Rancidness', 'Sourness', 'Spiciness', 'Sweetness', 'Unpalatability']
the property of having a harsh unpleasant taste


In [ ]:
sentence = "Bases are **good** conductors of electricity in the aqueous state."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Bases are good conductors of electricity in the aqueous state. answer: good </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Are base conductors of electricity in the aqueous state?
good
['Expedience', 'Favor', 'Favorableness', 'Handicap', 'Homecourt Advantage', 'Lead', 'Leverage', 'Preference', 'Privilege', 'Profit', 'Pull', 'Start', 'Superiority', 'Tax Advantage']
benefit


In [ ]:
sentence = "The pH of a solution is **inversely** proportional to the concentration of hydrogen ions in it."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The pH of a solution is inversely proportional to the concentration of hydrogen ions in it. answer: inversely </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


How is the pH of a solution proportional to the concentration of hydrogen ions?
inversely
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "The salts which have lost their water of crystallisation are known as **Anhydrous** salts"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The salts which have lost their water of crystallisation are known as Anhydrous salts answer: Anhydrous </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What are salts that have lost their water of crystallisation called?
Anhydrous
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "Metals are **ductile** which means that metals can be drawn or stretched into thin wires"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Metals are ductile which means that metals can be drawn or stretched into thin wires answer: ductile </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is it called when metals can be drawn or stretched into thin wires?
ductile
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "Non-metals have low **densities**, that is, non-metals are light substances."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Non-metals have low densities , that is, non-metals are light substances. answer: densities </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Non-metals have low what?
densities
['Density']
the amount per unit size


In [ ]:
sentence = "**Calcination** is the process in which a carbonate ore is heated strongly in the absence of air to convert it into metal oxide."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Calcination is the process in which a carbonate ore is heated strongly in the absence of air to convert it into metal oxide. answer: Calcination </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is it called when carbonate ore is heated in the absence of air to convert it into metal oxide?
Calcination
['Combustion', 'Nitrification', 'Rust']
the conversion of metals into their oxides as a result of heating to a high temperature


In [ ]:
sentence = "**Roasting** is the process in which a sulphide ore is strongly heated in the presence of air to convert it into metal oxide."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Roasting is the process in which a sulphide ore is strongly heated in the presence of air to convert it into metal oxide. answer: Roasting </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the process in which sulphide ore is heated in the presence of air to convert it into metal oxide?
Roasting
['Baking', 'Boiling', 'Braising', 'Broil', 'Cuisine', 'Frying', 'Fusion Cooking', 'Percolation', 'Poaching', 'Toasting']
cooking (meat) by dry heat in an oven (usually with fat added)


In [ ]:
sentence = "**Metallurgy** is defined as a process used for the extraction of metals in their pure form at low cost and minimum effort."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Metallurgy is defined as a process used for the extraction of metals in their pure form at low cost and minimum effort. answer: Metallurgy </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is a process used for the extraction of metals in their pure form?
Metallurgy
['Agrobiology', 'Agrology', 'Agronomy', 'Architectonics', 'Cognitive Science', 'Cryptanalysis', 'Information Science', 'Linguistics', 'Mathematics', 'Metrology', 'Natural History', 'Natural Science', 'Nutrition', 'Psychology', 'Social Science', 'Strategics', 'Systematics', 'Thanatology']
the science and technology of metals


In [ ]:
sentence = "Rust is a **red-brown** coloured substance."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Rust is a red-brown coloured substance. answer: red-brown </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What color is Rust?
red-brown
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "An **alloy** is prepared by mixing the various metals in the molten state in required proportions."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: An alloy is prepared by mixing the various metals in the molten state in required proportions. answer: alloy </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is prepared by mixing metals in molten state in required proportions?
alloy
['Colloid', 'Composition', 'Eutectic', 'Freezing Mixture', 'Greek Fire', 'Gummite', 'Lamellar Mixture', 'Matte', 'Mechanical Mixture', 'Oxyacetylene', 'Petrolatum', 'Plaster', 'Soda Lime', 'Solution', 'Suspension']
a mixture containing two or more metallic elements or metallic and nonmetallic elements usually fused together or dissolving into each other when molten


In [ ]:
sentence = " **Brass** is an alloy of Copper(Cu) and Zinc(Zn)"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Brass is an alloy of Copper(Cu) and Zinc(Zn) answer: Brass </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Copper(Cu) and Zinc(Zn) make up what alloy?
Brass
['Aluminum Bronze', 'Bronze', 'Constantan', 'Cupronickel']
an alloy of copper and zinc


In [ ]:
sentence = "**Bronze** is an alloy of Copper and Tin (Cu and Sn)."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Bronze is an alloy of Copper and Tin (Cu and Sn). answer: Bronze </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is an alloy of Copper and Tin?
Bronze
['Aluminum Bronze', 'Brass', 'Constantan', 'Cupronickel']
an alloy of copper and tin and sometimes other elements; also any copper-base alloy containing other elements in place of tin


In [ ]:
sentence = "**Solder** is an alloy of lead and tin (Pb and Sn)."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Solder is an alloy of lead and tin (Pb and Sn). answer: Solder </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is an alloy of lead and tin?
Solder
['18-karat Gold', '22-karat Gold', 'Alnico', 'Amalgam', 'Babbitt Metal', 'Britannia Metal', 'Carboloy', 'Cheoplastic Metal', 'Copper-base Alloy', 'Dental Gold', 'Duralumin', 'Electrum', 'Fusible Metal', 'Heavy Metal', 'Inconel', 'Invar', 'Nickel-base Alloy', 'Nickel Silver', 'Oroide', 'Pewter', 'Pinchbeck', 'Pot Metal', 'Pyrophoric Alloy', 'Shot Metal', 'Steel', 'Stellite', 'Sterling Silver', 'Tombac', 'Type Metal', 'White Gold', 'White Metal', "Wood's Metal"]
an alloy (usually of lead and tin) used when melted to join two metal surfaces


In [ ]:
sentence = "An alloy of mercury with one or more other metals is known as an **amalgam.**"



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: An alloy of mercury with one or more other metals is known as an amalgam. answer: amalgam. </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is an alloy of mercury with one or more other metals called?
amalgam.
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "Nervous system is made of special cells called **neurons**. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Nervous system is made of special cells called neurons . answer: neurons </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Nervous system is made up of special cells called what?
neurons
['Blood Cell', 'Bone Cell', 'Epithelial Cell', 'Fat Cell', 'Hybridoma', 'Mast Cell', 'Muscle Cell', 'Neoplastic Cell', 'Nerve Cell', 'Neurogliacyte', 'Phagocyte', 'Schwann Cell', 'Skin Cell', 'Stem Cell', 'Target Cell', 'Visual Cell']
a cell that is specialized to conduct nerve impulses


In [ ]:
sentence = "**Sensory** Neurons transmit impulses from the sensory cells or receptor towards the central nervous system which constitutes spinal cord and brain."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Sensory Neurons transmit impulses from the sensory cells or receptor towards the central nervous system which constitutes spinal cord and brain. answer: Sensory </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What neurons transmit impulses from the sensory cells to the central nervous system?
Sensory
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "**Synaps** is a microscopic gap between a pair of adjacent neurons over which nerve impulses pass when going from one neuron to the next."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Synaps is a microscopic gap between a pair of adjacent neurons over which nerve impulses pass when going from one neuron to the next. answer: Synaps </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is a microscopic gap between two adjacent neurons called?
Synaps
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "**Cranium** is a bony box which protects the brain."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Cranium is a bony box which protects the brain. answer: Cranium </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is a bony box that protects the brain?
Cranium
['Anklebone', 'Bare Bone', 'Calvaria', 'Cannon Bone', 'Carpal Bone', 'Cartilage Bone', 'Centrum', 'Cheekbone', 'Clavicle', 'Coccyx', 'Cuboid Bone', 'Dentine', 'Ethmoid', 'Fishbone', 'Furcula', 'Gladiolus', 'Heelbone', 'Hipbone', 'Hyoid', 'Ilium', 'Ischium', 'Jaw', 'Lacrimal Bone', 'Long Bone', 'Manubrium', 'Marrowbone', 'Membrane Bone', 'Metacarpal', 'Metatarsal', 'Modiolus', 'Nasal', 'Occiput', 'Ossicle', 'Palatine', 'Pastern', 'Phalanx', 'Pubis', 'Ramus', 'Rib', 'Round Bone', 'Sacrum', 'Scapula', 'Sesamoid Bone', 'Short Bone', 'Sinciput', 'Skull', 'Sphenoid Bone', 'Splint Bone', 'Sternum', 'Tarsal', 'Temporal Bone', 'Tooth', 'Turbinate Bone', 'Tympanic Bone', 'Vertebra', 'Vomer', 'Wormian Bone', 'Xiphoid Process', 'Zygoma']
the part of the skull that encloses the brain


In [ ]:
sentence = "The **fore-brain** is the main thinking part of the brain."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The fore-brain is the main thinking part of the brain. answer: fore-brain </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the main thinking part of the brain?
fore-brain
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "The **cerebral cortex** is the largest part of the brain and is divided into two further parts, i.e. the left hemisphere and the right hemisphere."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The cerebral cortex is the largest part of the brain and is divided into two further parts, i.e. the left hemisphere and the right hemisphere. answer: cerebral cortex </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the largest part of the brain?
cerebral cortex
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "The **amygdala** makes humans aware of their emotions and also responds to emotions appropriately."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The amygdala makes humans aware of their emotions and also responds to emotions appropriately. answer: amygdala </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What makes humans aware of their emotions?
amygdala
['Caudate Nucleus', 'Claustrum', 'Corpus Striatum', 'Lenticular Nucleus', 'Pallidum', 'Putamen']
an almond-shaped neural structure in the anterior part of the temporal lobe of the cerebrum; intimately connected with the hypothalamus and the hippocampus and the cingulate gyrus; as part of the limbic system it plays an important role in motivation and emotional behavior


In [ ]:
sentence = "The **thalamus** is responsible for realization that if one is asleep or awake."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The thalamus is responsible for realization that if one is asleep or awake. answer: thalamus </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What part of the brain determines if someone is awake or asleep?
thalamus
['Autonomic Nervous System', 'Brain', 'Brainstem', 'Center', 'Cerebellar Hemisphere', 'Cerebellum', 'Cerebral Cortex', 'Cerebrum', 'Cingulate Gyrus', 'Diencephalon', 'Forebrain', 'Ganglion', 'Geniculate Body', 'Hemisphere', 'Hindbrain', 'Hippocampus', 'Hypothalamus', 'Limbic System', 'Mamillary Body', 'Medulla Oblongata', 'Midbrain', 'Myelencephalon', 'Neencephalon', 'Nucleus', 'Olfactory Bulb', 'Paleencephalon', 'Paleocerebellum', 'Pons', 'Pyriform Area', 'Radiation', 'Reflex Arc', 'Spinal Cord', 'Substantia Nigra', 'Subthalamus', 'Telencephalon', 'Vermis']
large egg-shaped structures of grey matter that form the dorsal subdivision of the diencephalon


In [ ]:
sentence = "The **hypothalamus** is associated with activities that regulate activities such as eating and drinking."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)





context: The hypothalamus is associated with activities that regulate activities such as eating and drinking. answer: hypothalamus </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What part of the body regulates eating and drinking?
hypothalamus
['Autonomic Nervous System', 'Brain', 'Brainstem', 'Center', 'Cerebellar Hemisphere', 'Cerebellum', 'Cerebral Cortex', 'Cerebrum', 'Cingulate Gyrus', 'Diencephalon', 'Forebrain', 'Ganglion', 'Geniculate Body', 'Hemisphere', 'Hindbrain', 'Hippocampus', 'Limbic System', 'Mamillary Body', 'Medulla Oblongata', 'Midbrain', 'Myelencephalon', 'Neencephalon', 'Nucleus', 'Olfactory Bulb', 'Paleencephalon', 'Paleocerebellum', 'Pons', 'Pyriform Area', 'Radiation', 'Reflex Arc', 'Spinal Cord', 'Substantia Nigra', 'Subthalamus', 'Telencephalon', 'Thalamus', 'Vermis']
a basal part of the diencephalon governing autonomic nervous system


In [ ]:
sentence = "The **hippocampus** deals with memories."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The hippocampus deals with memories. answer: hippocampus </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What part of the brain deals with memories?
hippocampus
['Autonomic Nervous System', 'Brain', 'Brainstem', 'Center', 'Cerebellar Hemisphere', 'Cerebellum', 'Cerebral Cortex', 'Cerebrum', 'Cingulate Gyrus', 'Diencephalon', 'Forebrain', 'Ganglion', 'Geniculate Body', 'Hemisphere', 'Hindbrain', 'Hypothalamus', 'Limbic System', 'Mamillary Body', 'Medulla Oblongata', 'Midbrain', 'Myelencephalon', 'Neencephalon', 'Nucleus', 'Olfactory Bulb', 'Paleencephalon', 'Paleocerebellum', 'Pons', 'Pyriform Area', 'Radiation', 'Reflex Arc', 'Spinal Cord', 'Substantia Nigra', 'Subthalamus', 'Telencephalon', 'Thalamus', 'Vermis']
a complex neural structure (shaped like a sea horse) consisting of grey matter and located on the floor of each lateral ventricle; intimately involved in motivation and emotion as part of the limbic system; has a central role in the formation of memories


In [ ]:
sentence = "The **cerebrum** (or forebrain) is the main thinking part of the brain."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The cerebrum (or forebrain) is the main thinking part of the brain. answer: cerebrum </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the main thinking part of the brain?
cerebrum
['Autonomic Nervous System', 'Brain', 'Brainstem', 'Center', 'Cerebellar Hemisphere', 'Cerebellum', 'Cerebral Cortex', 'Cingulate Gyrus', 'Diencephalon', 'Forebrain', 'Ganglion', 'Geniculate Body', 'Hemisphere', 'Hindbrain', 'Hippocampus', 'Hypothalamus', 'Limbic System', 'Mamillary Body', 'Medulla Oblongata', 'Midbrain', 'Myelencephalon', 'Neencephalon', 'Nucleus', 'Olfactory Bulb', 'Paleencephalon', 'Paleocerebellum', 'Pons', 'Pyriform Area', 'Radiation', 'Reflex Arc', 'Spinal Cord', 'Substantia Nigra', 'Subthalamus', 'Telencephalon', 'Thalamus', 'Vermis']
anterior portion of the brain consisting of two hemispheres; dominant part of the brain in humans


In [ ]:
sentence = "**Medulla** controls various involuntary action such as heart beat (blood circulation), breathing, blood pressure and peristaltic movements of alimentary canal."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Medulla controls various involuntary action such as heart beat (blood circulation), breathing, blood pressure and peristaltic movements of alimentary canal. answer: Medulla </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What controls breathing, blood pressure, and peristaltic movements of the alimentary canal?
Medulla
['Autonomic Nervous System', 'Brain', 'Brainstem', 'Center', 'Cerebellar Hemisphere', 'Cerebellum', 'Cerebral Cortex', 'Cerebrum', 'Cingulate Gyrus', 'Diencephalon', 'Forebrain', 'Ganglion', 'Geniculate Body', 'Hemisphere', 'Hindbrain', 'Hippocampus', 'Hypothalamus', 'Limbic System', 'Mamillary Body', 'Medulla Oblongata', 'Midbrain', 'Myelencephalon', 'Neencephalon', 'Nucleus', 'Olfactory Bulb', 'Paleencephalon', 'Paleocerebellum', 'Pons', 'Pyriform Area', 'Radiation', 'Reflex Arc', 'Spinal Cord', 'Substantia Nigra', 'Subthalamus', 'Telencephalon', 'Thalamus', 'Vermis']
lower or hindmost part of the brain; continuous with spinal cord; (`bulb' is an old term for medulla oblongata)


In [ ]:
sentence = "The **pons** acts as a bridge with the midbrain and controls the intensity and frequency of breathing mechanism and sleep cycles."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The pons acts as a bridge with the midbrain and controls the intensity and frequency of breathing mechanism and sleep cycles. answer: pons </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What acts as a bridge with the midbrain?
pons
['Autonomic Nervous System', 'Brain', 'Brainstem', 'Center', 'Cerebellar Hemisphere', 'Cerebellum', 'Cerebral Cortex', 'Cerebrum', 'Cingulate Gyrus', 'Diencephalon', 'Forebrain', 'Ganglion', 'Geniculate Body', 'Hemisphere', 'Hindbrain', 'Hippocampus', 'Hypothalamus', 'Limbic System', 'Mamillary Body', 'Medulla Oblongata', 'Midbrain', 'Myelencephalon', 'Neencephalon', 'Nucleus', 'Olfactory Bulb', 'Paleencephalon', 'Paleocerebellum', 'Pyriform Area', 'Radiation', 'Reflex Arc', 'Spinal Cord', 'Substantia Nigra', 'Subthalamus', 'Telencephalon', 'Thalamus', 'Vermis']
a band of nerve fibers linking the medulla oblongata and the cerebellum with the midbrain


In [ ]:
sentence = "The **skull** protects the brain by providing a bony structure."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The skull protects the brain by providing a bony structure. answer: skull </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What protects the brain by providing a bony structure?
skull
['Anklebone', 'Bare Bone', 'Calvaria', 'Cannon Bone', 'Carpal Bone', 'Cartilage Bone', 'Centrum', 'Cheekbone', 'Clavicle', 'Coccyx', 'Cranium', 'Cuboid Bone', 'Dentine', 'Ethmoid', 'Fishbone', 'Furcula', 'Gladiolus', 'Heelbone', 'Hipbone', 'Hyoid', 'Ilium', 'Ischium', 'Jaw', 'Lacrimal Bone', 'Long Bone', 'Manubrium', 'Marrowbone', 'Membrane Bone', 'Metacarpal', 'Metatarsal', 'Modiolus', 'Nasal', 'Occiput', 'Ossicle', 'Palatine', 'Pastern', 'Phalanx', 'Pubis', 'Ramus', 'Rib', 'Round Bone', 'Sacrum', 'Scapula', 'Sesamoid Bone', 'Short Bone', 'Sinciput', 'Sphenoid Bone', 'Splint Bone', 'Sternum', 'Tarsal', 'Temporal Bone', 'Tooth', 'Turbinate Bone', 'Tympanic Bone', 'Vertebra', 'Vomer', 'Wormian Bone', 'Xiphoid Process', 'Zygoma']
the bony skeleton of the head of vertebrates


In [ ]:
sentence = "The **spinal cord** begins in continuation with medulla and extends downwards. It is enclosed in a bony cage called vertebral column."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The spinal cord begins in continuation with medulla and extends downwards. It is enclosed in a bony cage called vertebral column. answer: spinal cord </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What part of the body begins with medulla?
spinal cord
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "**Parathyroid** gland is a butterfly-shaped paired gland and releases parathormone which regulates calcium and phosphorus level in bones."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Parathyroid gland is a butterfly-shaped paired gland and releases parathormone which regulates calcium and phosphorus level in bones. answer: Parathyroid </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What gland releases parathormone?
Parathyroid
['Adrenal Cortex', 'Adrenal Gland', 'Adrenal Medulla', 'Anterior Pituitary', 'Corpus Luteum', 'Gonad', 'Islands Of Langerhans', 'Parathyroid Gland', 'Pars Distilis', 'Pars Intermedia', 'Pineal Gland', 'Pituitary', 'Posterior Pituitary', 'Prostate Gland', 'Thymus Gland', 'Thyroid Gland']
any one of four endocrine glands situated above or within the thyroid gland


In [ ]:
sentence = "The **adrenaline** hormone prepares our body to function at maximum efficiency during emergency situations like danger, anger, excitement, etc. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The adrenaline hormone prepares our body to function at maximum efficiency during emergency situations like danger, anger, excitement, etc. answer: adrenaline </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What hormone prepares our body to function at maximum efficiency during emergencies?
adrenaline
['Epinephrine', 'Noradrenaline']
a catecholamine secreted by the adrenal medulla in response to stress (trade name Adrenalin); stimulates autonomic nerve action


In [ ]:
sentence = " **Pancreas** is endocrine as well as an exocrine gland. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Pancreas is endocrine as well as an exocrine gland. answer: Pancreas </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What gland is endocrine?
Pancreas
['Cervical Glands', "Cowper's Gland", 'Digestive Gland', 'Lacrimal Gland', 'Mammary Gland', 'Nabothian Gland', 'Oil Gland', 'Salivary Gland', 'Seminal Vesicle', 'Sweat Gland', 'Vestibular Gland']
a large elongated exocrine gland located behind the stomach; secretes pancreatic juice and insulin


In [ ]:
sentence = "**Chromosomes** are thread-like structure in the nucleus of a cell formed of DNA. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Chromosomes are thread-like structure in the nucleus of a cell formed of DNA. answer: Chromosomes </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the name of the thread-like structure in the nucleus of a cell formed of DNA?
Chromosomes
['Chromosome', 'Inclusion Body', 'Mass', 'Particle']
a threadlike strand of DNA in the cell nucleus that carries the genes in a linear order


In [ ]:
sentence = " **Genes** are units of heredity which transfer characteristics (or traits) from parents to their offsprings."



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Genes are units of heredity which transfer characteristics (or traits) from parents to their offsprings. answer: Genes </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What are units of heredity that transfer traits from parents to their offspring?
Genes
['Codon', 'Combination', 'Fibonacci Sequence', 'Gene', 'String']
(genetics) a segment of DNA that is involved in producing a polypeptide chain; it can include regions preceding and following the coding DNA as well as introns between the exons; it is considered a unit of heredity


In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Phenotype is the characteristic (or traits) which is visible in an organism. answer: Phenotype </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the characteristic that is visible in an organism?
Phenotype
['Genotype', 'Karyotype', 'Structure', 'Texture']
what an organism looks like as a consequence of the interaction of its genotype and the environment


In [ ]:
sentence = "According to **wave** theory, light consists of electromagnetic waves which do not require a material medium (like solid, liquid or gas) for their propagation. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: According to wave theory, light consists of electromagnetic waves which do not require a material medium (like solid, liquid or gas) for their propagation. answer: wave </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What theory states that light consists of electromagnetic waves?
wave
['Approach', 'Bending', 'Brownian Movement', 'Change Of Location', 'Crustal Movement', 'Deflection', 'Heave', 'Jitter', 'Passing', 'Periodic Motion', 'Recoil', 'Seek', 'Squeeze', 'Throw', 'Turning', 'Twist', 'Undulation', 'Whirl', 'Wobble']
(physics) a movement up and down or back and forth


In [ ]:
sentence = " According to **particle theory**, light is composed of particles which travel in a straight line at very high speed "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: According to particle theory , light is composed of particles which travel in a straight line at very high speed answer: particle theory </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


Light is composed of particles that travel in a straight line at high speed according to what theory?
particle theory
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " When light falls on a polished smooth surface such as mirror, metal surface and clear water, it bounces back and the phenomena is called **reflection** of light. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: When light falls on a polished smooth surface such as mirror, metal surface and clear water, it bounces back and the phenomena is called reflection of light. answer: reflection </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the phenomenon called when light bounces back?
reflection
['Acoustic Phenomenon', 'Atmospheric Phenomenon', 'Boundary Layer', 'Chaos', 'Chop', 'Cloud', 'Conduction', 'Decalescence', 'Electrical Phenomenon', 'Electricity', 'Energy', 'Event', 'Field', 'Flotation', 'Force', 'Hysteresis', 'Interaction', 'Mechanical Phenomenon', 'Opacity', 'Optical Phenomenon', 'Power', 'Pressure', 'Propagation', 'Refraction', 'Resolution', 'Resolving Power', 'Resonance', 'Surface Tension', 'Syzygy', 'Transparency', 'Turbulence']
the phenomenon of a propagating wave (light or sound) being thrown back from a surface


In [ ]:
sentence = " The ray of light which falls on the polished surface is called incident ray and the ray that bounces back is called **reflected** ray. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The ray of light which falls on the polished surface is called incident ray and the ray that bounces back is called reflected ray. answer: reflected </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the ray that bounces back called?
reflected
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " The angle of incidence (<i) is **equal** to the angle of reflection(<r).  "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The angle of incidence (<i) is equal to the angle of reflection(<r). answer: equal </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the relationship between the angle of incidence and reflection?
equal
['Abator', 'Abjurer', 'Abomination', 'Abstainer', 'Achiever', 'Acquaintance', 'Acquirer', 'Active', 'Actor', 'Adjudicator', 'Admirer', 'Adoptee', 'Adult', 'Adventurer', 'Adversary', 'Advisee', 'Advocate', 'Affiant', 'African', 'Agnostic', 'Amateur', 'Amerindian', 'Ancient', 'Anomaly', 'Anti-american', 'Anti', 'Applicant', 'Appointee', 'Appreciator', 'Apprehender', 'Aquarius', 'Archaist', 'Aries', 'Arrogator', 'Assessee', 'Asthmatic', 'Authority', 'Autodidact', 'Baby Boomer', 'Baby Buster', 'Bad Guy', 'Bad Person', 'Baldhead', 'Balker', 'Bather', 'Beard', 'Bedfellow', 'Bereaved', 'Best', 'Birth', 'Biter', 'Black', 'Blogger', 'Blond', 'Bluecoat', 'Bodybuilder', 'Bomber', 'Brunet', 'Bullfighter', 'Buster', 'Cancer', 'Candidate', 'Capitalist', 'Capricorn', 'Captor', 'Case', 'Cashier', 'Celebrant', 'Censor', 'Chameleon', 'Changer', 'Charmer', 'Child', 'Chutzpanik', 'Closer', 'Clumsy Person', 'Collector', 'Color-bl

In [ ]:
sentence = "  When a parallel beam of light falls on the smooth surface of a plane mirror, it is reflected as a **parallel** beam. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: When a parallel beam of light falls on the smooth surface of a plane mirror, it is reflected as a parallel beam. answer: parallel </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What type of beam of light is reflected on a plane mirror?
parallel
['Equilateral', 'Pencil', 'Plane Figure', 'Solid Figure', 'Subfigure']
(mathematics) one of a set of parallel geometric figures (parallel lines or planes)


In [ ]:
sentence = " A real image is **inverted** "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: A real image is inverted answer: inverted </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


A real image is what?
inverted
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " In **lateral inversion**, the right side of the object appearsleft side of the image and viceversa. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: In lateral inversion , the right side of the object appearsleft side of the image and viceversa. answer: lateral inversion </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


In what type of inversion does the right side of an object appear on the other side?
lateral inversion
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = "**Centre Of Curvature** is the Centre of the hollow sphere of glass of which the mirror is a part. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Centre Of Curvature is the Centre of the hollow sphere of glass of which the mirror is a part. answer: Centre Of Curvature </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the centre of the hollow sphere of glass of which the mirror is a part?
Centre Of Curvature
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " **Pole** is the centre of a spherical mirror. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Pole is the centre of a spherical mirror. answer: Pole </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the center of a spherical mirror?
Pole
['Baton', 'Connecting Rod', 'Control Rod', 'Fishing Rod', 'Kickstand', 'Ramrod', 'Rotating Shaft', 'Shaft', 'Stair-rod', 'Tie Rod', 'Wand']
a long (usually round) rod of wood or metal or plastic


In [ ]:
sentence = " **Principle Axis** is the straight line passing through the centre of curvature and pole of a spherical mirror. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Principle Axis is the straight line passing through the centre of curvature and pole of a spherical mirror. answer: Principle Axis </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the straight line passing through the centre of curvature and pole of a spherical mirror?
Principle Axis
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " The change in direction of light when it passes from one medium to another obliquely is called **refraction** of light. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The change in direction of light when it passes from one medium to another obliquely is called refraction of light. answer: refraction </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the change in direction of light when it passes from one medium to another obliquely called?
refraction
['Acoustic Phenomenon', 'Atmospheric Phenomenon', 'Boundary Layer', 'Chaos', 'Chop', 'Cloud', 'Conduction', 'Decalescence', 'Electrical Phenomenon', 'Electricity', 'Energy', 'Event', 'Field', 'Flotation', 'Force', 'Hysteresis', 'Interaction', 'Mechanical Phenomenon', 'Opacity', 'Optical Phenomenon', 'Power', 'Pressure', 'Propagation', 'Reflection', 'Resolution', 'Resolving Power', 'Resonance', 'Surface Tension', 'Syzygy', 'Transparency', 'Turbulence']
the change in direction of a propagating wave (light or sound) when passing from one medium to another


In [ ]:
sentence = " A medium in which the speed of light is more is known as optically **rarer** medium.  "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: A medium in which the speed of light is more is known as optically rarer medium. answer: rarer </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is an optically rarer medium called?
rarer
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " The second law of refraction is called **Snell’s law of refraction.** "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The second law of refraction is called Snell’s law of refraction. answer: Snell’s law of refraction. </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the second law of refraction called?
Snell’s law of refraction.
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " The front part of the eye is called **cornea**. It is made of a transparent substance. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The front part of the eye is called cornea . It is made of a transparent substance. answer: cornea </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the front part of the eye called?
cornea
['Axolemma', 'Basilar Membrane', "Bowman's Capsule", 'Chorioallantois', 'Choroid', 'Ciliary Body', 'Diaphragm', 'Eardrum', 'Endocranium', 'Endosteum', 'Ependyma', 'Fertilization Membrane', 'Fetal Membrane', 'Hyaloid Membrane', 'Intima', 'Iris', 'Lamella', 'Meninx', 'Mucous Membrane', 'Periosteum', 'Perithelium', 'Retina', 'Sarcolemma', 'Semipermeable Membrane', 'Serous Membrane', 'Synovial Membrane', 'Trophoblast', 'Tunic', 'Web']
the transparent dome-shaped anterior portion of the outer covering of the eye; it covers the iris and pupil and is continuous with the sclera


In [ ]:
sentence = " Most of the **refraction** (bending) for the light rays entering the eye occurs at the outer surface of the cornea. It acts as a window for the human eye and its role is to prevent foreign substances from entering the eye "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Most of the refraction (bending) for the light rays entering the eye occurs at the outer surface of the cornea. It acts as a window for the human eye and its role is to prevent foreign substances from entering the eye answer: refraction </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the term for the bending of light rays entering the eye?
refraction
['Acoustic Phenomenon', 'Atmospheric Phenomenon', 'Boundary Layer', 'Chaos', 'Chop', 'Cloud', 'Conduction', 'Decalescence', 'Electrical Phenomenon', 'Electricity', 'Energy', 'Event', 'Field', 'Flotation', 'Force', 'Hysteresis', 'Interaction', 'Mechanical Phenomenon', 'Opacity', 'Optical Phenomenon', 'Power', 'Pressure', 'Propagation', 'Reflection', 'Resolution', 'Resolving Power', 'Resonance', 'Surface Tension', 'Syzygy', 'Transparency', 'Turbulence']
the change in direction of a propagating wave (light or sound) when passing from one medium to another


In [ ]:
sentence = " **Iris** is like a camera shutter that controls the amount of light entering the eye.. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Iris is like a camera shutter that controls the amount of light entering the eye.. answer: Iris </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What controls the amount of light entering the eye?
Iris
['Axolemma', 'Basilar Membrane', "Bowman's Capsule", 'Chorioallantois', 'Choroid', 'Ciliary Body', 'Cornea', 'Diaphragm', 'Eardrum', 'Endocranium', 'Endosteum', 'Ependyma', 'Fertilization Membrane', 'Fetal Membrane', 'Hyaloid Membrane', 'Intima', 'Lamella', 'Meninx', 'Mucous Membrane', 'Periosteum', 'Perithelium', 'Retina', 'Sarcolemma', 'Semipermeable Membrane', 'Serous Membrane', 'Synovial Membrane', 'Trophoblast', 'Tunic', 'Web']
muscular diaphragm that controls the size of the pupil which in turn controls the amount of light that enters the eye; it forms the colored portion of the eye


In [ ]:
sentence = "The unit power of accommodation for a normal eye is **Dioptre** "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The unit power of accommodation for a normal eye is Dioptre answer: Dioptre </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the unit power of accommodation for a normal eye?
Dioptre
['Absorption Unit', 'Acceleration Unit', 'Angular Unit', 'Area Unit', 'Bit', 'Brinell Number', 'Computer Memory Unit', 'Denier', 'Diopter', 'Electromagnetic Unit', 'Electrostatic Unit', 'Explosive Unit', 'Force Unit', 'Karat', 'Langley', 'Linear Unit', 'Mass Unit', 'Measuring Unit', 'Megaflop', 'Metric Unit', 'Miles Per Gallon', 'Mips', 'Monetary Unit', 'Pain Unit', 'Point', 'Pressure Unit', 'Printing Unit', 'Sound Unit', 'Telephone Unit', 'Temperature Unit', 'Teraflop', 'Unit Of Viscosity', 'Volume Unit', 'Weight Unit', 'Work Unit']
a unit of measurement of the refractive power of a lens which is equal to the reciprocal of the focal length measured in meters; used by oculists


In [ ]:
sentence = " The phenomenon of the continuation of the impression of an image on the retina for sometime even after the light from the object is cut off is called **persistence of vision.** "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: The phenomenon of the continuation of the impression of an image on the retina for sometime even after the light from the object is cut off is called persistence of vision. answer: persistence of vision. </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is the phenomenon of continuing the impression of an image on the retina even after the light from an object is cut off called?
persistence of vision.
['Word not found in Wordnet. So unable to extract distractors.']
None


In [ ]:
sentence = " **Myopia** is also known as near-sightedness or short sightedness. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Myopia is also known as near-sightedness or short sightedness. answer: Myopia </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is another name for near-sightedness?
Myopia
['Astigmatism', 'Hyperopia']
(ophthalmology) eyesight abnormality resulting from the eye's faulty refractive ability; distant objects appear blurred


In [ ]:
sentence = " **Hypermetropia** is also known as long-sightedness or far-sightedness. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Hypermetropia is also known as long-sightedness or far-sightedness. answer: Hypermetropia </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is another name for long-sightedness?
Hypermetropia
['Astigmatism', 'Hyperopia', 'Myopia']
abnormal condition in which vision for distant objects is better than for near objects


In [ ]:
sentence = " **Presbyopia** is that defect of vision due to which an old person cannot see the nearby objects clearly due to loss of power of accommodation of the eye. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Presbyopia is that defect of vision due to which an old person cannot see the nearby objects clearly due to loss of power of accommodation of the eye. answer: Presbyopia </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is it called when an old person can't see objects clearly?
Presbyopia
[]
a reduced ability to focus on near objects caused by loss of elasticity of the crystalline lens after age 45


In [ ]:
sentence = " **Galvanometer** is a device to detect current in an electric circuit. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Galvanometer is a device to detect current in an electric circuit. answer: Galvanometer </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is a device to detect current in an electric circuit called?
Galvanometer
['Ammeter', 'Electric Meter', 'Electrometer', 'Fluxmeter', 'Gas Meter', 'Magnetometer', 'Odometer', 'Ohmmeter', 'Postage Meter', 'Radiometer', 'Reflectometer', 'Speedometer', 'Taximeter', 'Voltmeter', 'Volumeter', 'Water Meter']
meter for detecting or comparing or measuring small electric currents


In [ ]:
sentence = " **Voltmeter** is used to measure the voltage between two points in a circuit. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Voltmeter is used to measure the voltage between two points in a circuit. answer: Voltmeter </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is used to measure the voltage between two points in a circuit?
Voltmeter
['Ammeter', 'Electric Meter', 'Electrometer', 'Fluxmeter', 'Galvanometer', 'Gas Meter', 'Magnetometer', 'Odometer', 'Ohmmeter', 'Postage Meter', 'Radiometer', 'Reflectometer', 'Speedometer', 'Taximeter', 'Volumeter', 'Water Meter']
meter that measures the potential difference between two points


In [ ]:
sentence = " **Ammeter** is used to measure the current passing at a particular point. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Ammeter is used to measure the current passing at a particular point. answer: Ammeter </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is used to measure the current passing at a particular point?
Ammeter
['Electric Meter', 'Electrometer', 'Fluxmeter', 'Galvanometer', 'Gas Meter', 'Magnetometer', 'Odometer', 'Ohmmeter', 'Postage Meter', 'Radiometer', 'Reflectometer', 'Speedometer', 'Taximeter', 'Voltmeter', 'Volumeter', 'Water Meter']
a meter that measures the flow of electrical current in amperes


In [ ]:
sentence = " **Resistor** is used to restrict the amount of current flow in the circuit. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Resistor is used to restrict the amount of current flow in the circuit. answer: Resistor </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is used to restrict the amount of current flow in a circuit?
Resistor
['Antenna', 'Attenuator', 'Ballast', 'Battery', 'Bimetallic Strip', 'Brush', 'Capacitor', 'Cell', 'Circuit', 'Control Panel', 'Distributor', 'Electrograph', 'Electrostatic Generator', 'Filter', 'Flasher', 'Fuse', 'Inductor', 'Jack', 'Load', 'Plug', 'Precipitator', 'Reactor', 'Rectifier', 'Relay', 'Security System', 'Segway', 'Solar Array', 'Spark Arrester', 'Spark Plug', 'Suppressor', 'Transducer', 'Transformer', 'Transponder', 'Zapper']
an electrical device that resists the flow of electrical current


In [ ]:
sentence = " **Battery** is a combination of cells and its utility is same as the cell. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)



context: Battery is a combination of cells and its utility is same as the cell. answer: Battery </s>


/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


What is a combination of cells called?
Battery
['Antenna', 'Attenuator', 'Ballast', 'Bimetallic Strip', 'Brush', 'Capacitor', 'Cell', 'Circuit', 'Control Panel', 'Distributor', 'Electrograph', 'Electrostatic Generator', 'Filter', 'Flasher', 'Fuse', 'Inductor', 'Jack', 'Load', 'Plug', 'Precipitator', 'Reactor', 'Rectifier', 'Relay', 'Resistor', 'Security System', 'Segway', 'Solar Array', 'Spark Arrester', 'Spark Plug', 'Suppressor', 'Transducer', 'Transformer', 'Transponder', 'Zapper']
a device that produces electricity; may have several primary or secondary cells arranged in parallel or series


In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
sentence = " **Phenotype** is the characteristic (or traits) which is visible in an organism. "



print ("\n")
question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)